In [1]:
import numpy as np
import tensorflow as tf
import random
import sys
import eli5

import d2d
import importlib
importlib.reload(d2d)
import pandas as pd
import os

2022-11-16 10:23:23.416982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 10:23:23.546942: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-16 10:23:23.578441: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 10:23:24.152821: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
seeds = list(np.arange(1,101,1))


history = {}
val_performance = {}
performance = {}

residuals = {}
true_discharge_vals = {}

In [ ]:
for SEED in seeds:
    
    file = "/data/fast0/datasets/Rhone_data_continuous_highpass.h5"

    filt = 'Highpass'

    #window_width = [1,2,5, 200]
    #window_width = [350, 400, 450, 500]
    ww = 200
    bs = 32

    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)



    linear_model, lstm_model, dnn_model, df_all_chan, das_data_all, f = d2d.import_data(filename = file)

    da = df_all_chan.to_numpy()

    multi_step_window_shuffled = d2d.WindowGenerator(df_all_chan,
                                                    input_width=ww,
                                                    label_width=1, 
                                                    shift=0,
                                                    label_columns=['Discharge'],
                                                    input_columns=list(np.arange(0,2308,1)),
                                                    shuffle=True,
                                                    batch_size=bs)
    print("Fold number: "+str(SEED))
    
    history['lstm_'+str(ww)+'_'+filt+'_Fold:'+str(SEED)] = d2d.compile_and_fit(lstm_model, multi_step_window_shuffled, learning_rate = 0.001)

    val_performance['lstm_'+str(ww)+'_'+filt+'_Fold:'+str(SEED)] = lstm_model.evaluate(multi_step_window_shuffled.val)
    performance['lstm_'+str(ww)+'_'+filt+'_Fold:'+str(SEED)] = lstm_model.evaluate(multi_step_window_shuffled.test, verbose=0)
    
    pred_on_test = lstm_model.predict(multi_step_window_shuffled.test).squeeze()
    pred_on_test = np.asarray(pred_on_test)
    #pred_on_test = pred_on_test[:,-1]
    predictions_test = (pred_on_test * multi_step_window_shuffled.dis_std) + multi_step_window_shuffled.dis_mean
    
    test_true_dis = []

    for i in multi_step_window_shuffled.test.as_numpy_iterator():
        test_true_dis.append(i[1])

    test_true_dis = (test_true_dis[0].squeeze() * multi_step_window_shuffled.dis_std) + multi_step_window_shuffled.dis_mean
    
    residuals['Run ' + str(SEED)] = predictions_test - test_true_dis
    true_discharge_vals['Run ' + str(SEED)] = test_true_dis

In [ ]:
trueD = []
resD = []

for i, j in zip(residuals.values(), true_discharge_vals.values()):
    #ax.scatter(j , i, facecolor = 'None', edgecolor='steelblue')
    resD.append(i)
    trueD.append(j)

In [ ]:
trueD = np.asarray(trueD)
resD = np.asarray(resD)
trueD = np.reshape(trueD, trueD.shape[0]*trueD.shape[1])
resD = np.reshape(resD, resD.shape[0]*resD.shape[1])

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
fig.patch.set_facecolor('white')

c = ax.hist2d(trueD, resD, bins=[18,18])
ax.set_ylabel('Residual (m$^3$/s)', size = 12)
ax.set_xlabel('True Discharge (m$^3$/s)', size = 12)
ax.grid()

cbar = fig.colorbar(c[3], ax=ax)
cbar.set_label('Count', rotation=270)

#fig.savefig('LSTM_bootstrap_testSet_heatmap.eps')
plt.show()